In [74]:
import pandas as pd
import numpy as np
import datetime

In [307]:
fire = pd.read_csv("firesCA_cty.csv")
weather = pd.read_csv("weather_noNA.csv")
fire["DATE"] = pd.to_datetime(fire["DATE"])
weather["date"] = pd.to_datetime(weather["date"])

In [308]:
fire["FIPS_CODE"] = fire["FIPS_CODE"] % 6000

In [325]:
#bigsurvival = pd.DataFrame()

newcols=['fire_size','max_drybulbtemp','min_drybulbtemp','max_atm_moisture','min_atm_moisture',"max_avg_windspeed","min_avg_windspeed","max_fuelmoisture","min_fuelmoisture","max_maxtemp","min_mintemp","max_maxhumid","min_minhumid","sum_precip_duration","sum_precip_amt","med_precip_amt"]

for code in fire.FIPS_CODE.unique()[40:]:

    fire1 = (fire[fire.FIPS_CODE == code]).sort_values(by="DATE")
    try:
        fire1["NEXTDATE"] = list(fire1["DATE"].values)[1:] + [np.nan]
        fire1["SurvivalTime"] = (fire1["NEXTDATE"] - fire1["DATE"]).dt.days
        fire1["FireIncidence"] = 1
        survdata = fire1[["FOD_ID","FIPS_NAME","FIPS_CODE","DATE","NEXTDATE","SurvivalTime","FireIncidence"]]
    except:
        continue



    for i in newcols:
        survdata[i] = np.nan

    for j in range(len(survdata.index)):
        idx = survdata.index[j]
        if j==0:
            continue
        else:
            try:
                fireid = survdata.loc[idx,"FOD_ID"]
                survdata.loc[idx,"fire_size"] = fire.query("FOD_ID==@fireid").FIRE_SIZE.values[0]
                date1 = survdata.loc[survdata.index[j-1],"DATE"]
                date2 = survdata.loc[survdata.index[j-1],"NEXTDATE"]
                w=weather.query("date>@date1 and date<=@date2 and FIPS_CODE == @code")

                survdata.loc[idx,"max_drybulbtemp"] = w.drybulbtemp.max()
                survdata.loc[idx,"min_drybulbtemp"] = w.drybulbtemp.min()
                survdata.loc[idx,"max_atm_moisture"] = w.atm_moisture.max()
                survdata.loc[idx,"min_atm_moisture"] = w.atm_moisture.min()
                survdata.loc[idx,"max_avg_windspeed"] = w.avg_windspeed.max()
                survdata.loc[idx,"min_avg_windspeed"] = w.avg_windspeed.min()
                survdata.loc[idx,"max_fuelmoisture"] = w.fuelmoisture.max()
                survdata.loc[idx,"min_fuelmoisture"] = w.fuelmoisture.min()
                survdata.loc[idx,"max_maxtemp"] = w.maxtemp.max()
                survdata.loc[idx,"min_mintemp"] = w.mintemp.min()
                survdata.loc[idx,"max_maxhumid"] = w.maxhumid.max()
                survdata.loc[idx,"min_minhumid"] = w.minhumid.min()
                survdata.loc[idx,"sum_precip_duration"] = w.precip_duration.sum()
                survdata.loc[idx,"sum_precip_amt"] = w.precip_amt.sum()
                survdata.loc[idx,"med_precip_amt"] = w.precip_amt.median()
            except:
                continue
            
    bigsurvival = pd.concat([bigsurvival,survdata])
            


<ipython-input-325-3386462c1c99>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survdata[i] = np.nan


In [317]:
fireid = 201812071
idx = survdata.index[1]


In [319]:
survdata.loc[idx,"fire_size"] = fire.query("FOD_ID==@fireid").FIRE_SIZE.values[0]

In [314]:
survdata.loc[idx,"fire_size"]

nan

In [327]:
bigsurvival.dropna().to_csv("survivalv2.csv",index=False)

In [329]:
bigsurvival.dropna()

,FOD_ID,FIPS_NAME,FIPS_CODE,DATE,NEXTDATE,SurvivalTime,FireIncidence,fire_size,max_drybulbtemp,min_drybulbtemp,...,min_avg_windspeed,max_fuelmoisture,min_fuelmoisture,max_maxtemp,min_mintemp,max_maxhumid,min_minhumid,sum_precip_duration,sum_precip_amt,med_precip_amt
116118,1361688,Placer,61,1993-04-27,1993-04-30,3.0,1,0.3,48.0,40.0,...,3.0,22.0,9.0,49.0,28.0,100.0,45.0,0.0,0.0,0.0
115058,1360401,Placer,61,1993-05-10,1993-05-10,0.0,1,150.0,72.0,72.0,...,6.0,9.0,9.0,72.0,45.0,64.0,30.0,0.0,0.0,0.0
113650,1358702,Placer,61,1993-05-22,1993-05-23,1.0,1,20.0,74.0,69.0,...,2.0,7.0,7.0,74.0,50.0,65.0,38.0,0.0,0.0,0.0
115060,1360403,Placer,61,1993-05-24,1993-05-26,2.0,1,0.1,66.0,66.0,...,5.0,9.0,9.0,70.0,48.0,81.0,44.0,0.0,0.0,0.0
113651,1358703,Placer,61,1993-05-26,1993-05-26,0.0,1,2.0,46.0,46.0,...,9.0,25.0,25.0,68.0,44.0,100.0,46.0,3.0,510.0,510.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90882,1330668,Santa Cruz,87,1995-10-15,1995-10-18,3.0,1,1.5,69.0,69.0,...,4.0,12.0,12.0,71.0,56.0,100.0,58.0,0.0,0.0,0.0
90820,1330605,Santa Cruz,87,1995-10-18,1995-10-22,4.0,1,30.0,67.0,67.0,...,6.0,7.0,7.0,67.0,50.0,100.0,55.0,0.0,0.0,0.0
91095,1330885,Santa Cruz,87,1998-05-22,1998-07-16,55.0,1,0.1,67.0,65.0,...,4.0,26.0,13.0,77.0,51.0,100.0,53.0,3.0,90.0,45.0
91123,1330913,Santa Cruz,87,1998-07-16,1998-07-23,7.0,1,0.3,79.0,75.0,...,3.0,93.0,14.0,86.0,49.0,100.0,46.0,0.0,0.0,0.0


In [315]:
fire.query("FOD_ID==@fireid").FIRE_SIZE

165016    0.25
Name: FIRE_SIZE, dtype: float64

In [302]:
date1 = survdata.loc[survdata.index[400],"DATE"]
date2 = survdata.loc[survdata.index[1000],"NEXTDATE"]
w=weather.query("date>@date1 and date<=@date2")
w

,date,drybulbtemp,atm_moisture,wind_azimuth,avg_windspeed,fuelmoisture,maxtemp,mintemp,maxhumid,minhumid,precip_duration,precip_amt,moisture_type,meas_type,county,wetflag,snowflag,FIPS_NAME,FIPS_CODE
5,1993-05-19,73.0,37.0,203.0,4.0,6.0,74.0,44.0,98.0,32.0,0.0,0.0,2.0,1.0,017-ElDorado,N,N,ElDorado,17
6,1993-05-31,46.0,100.0,338.0,2.0,26.0,68.0,45.0,100.0,39.0,12.0,1390.0,2.0,1.0,017-ElDorado,Y,N,ElDorado,17
7,1993-07-04,80.0,30.0,315.0,2.0,5.0,80.0,45.0,72.0,22.0,0.0,0.0,2.0,1.0,017-ElDorado,N,N,ElDorado,17
8,1993-07-12,81.0,27.0,203.0,8.0,6.0,86.0,48.0,72.0,27.0,0.0,0.0,2.0,1.0,017-ElDorado,N,N,ElDorado,17
9,1993-07-16,72.0,27.0,338.0,5.0,5.0,74.0,39.0,89.0,21.0,0.0,0.0,2.0,1.0,017-ElDorado,N,N,ElDorado,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63156,1994-08-20,82.0,19.0,225.0,7.0,5.0,83.0,66.0,32.0,19.0,0.0,0.0,2.0,1.0,115-Yuba,N,N,Yuba,115
63157,1994-08-28,76.0,29.0,225.0,8.0,5.0,83.0,60.0,30.0,18.0,0.0,0.0,2.0,1.0,115-Yuba,N,N,Yuba,115
63158,1994-09-11,65.0,39.0,248.0,7.0,7.0,68.0,50.0,57.0,35.0,0.0,0.0,2.0,1.0,115-Yuba,N,N,Yuba,115
63159,1994-09-16,83.0,19.0,257.0,9.0,4.0,84.0,69.0,32.0,16.0,0.0,0.0,2.0,1.0,115-Yuba,N,N,Yuba,115


In [304]:
date2

Timestamp('1994-09-28 00:00:00')

In [279]:
survdata.index[1:]

Int64Index([117861, 117862, 117000, 118607, 117863, 117864, 117865, 118927,
            118161, 118928,
            ...
            180905, 182796, 181036, 180927, 180811, 181326, 181064, 180987,
            181052, 181042],
           dtype='int64', length=6798)

In [257]:
w=weather.query("date>'2013-03-11' and date<='2013-03-13' and FIPS_CODE == 89")
w

,date,drybulbtemp,atm_moisture,wind_azimuth,avg_windspeed,fuelmoisture,maxtemp,mintemp,maxhumid,minhumid,precip_duration,precip_amt,moisture_type,meas_type,county,wetflag,snowflag,FIPS_NAME,FIPS_CODE
59212,2013-03-12,70.1,34.6,194.6,3.3,8.0,73.2,44.5,70.3,25.4,0.0,0.0,2.0,1.0,089-Shasta,N,N,Shasta,89
59213,2013-03-13,68.9,39.7,156.2,3.2,8.5,71.4,45.0,76.0,31.5,0.0,0.0,2.0,1.0,089-Shasta,N,N,Shasta,89


In [264]:
survdata.loc[survdata["FOD_ID"]==201812071,"max_drybulbtemp"] = w.drybulbtemp.max()
survdata.loc[survdata["FOD_ID"]==201812071,"min_drybulbtemp"] = w.drybulbtemp.min()
survdata.loc[survdata["FOD_ID"]==201812071,"max_atm_moisture"] = w.atm_moisture.max()
survdata.loc[survdata["FOD_ID"]==201812071,"min_atm_moisture"] = w.atm_moisture.min()
survdata.loc[survdata["FOD_ID"]==201812071,"max_avg_windspeed"] = w.avg_windspeed.max()
survdata.loc[survdata["FOD_ID"]==201812071,"min_avg_windspeed"] = w.avg_windspeed.min()
survdata.loc[survdata["FOD_ID"]==201812071,"max_fuelmoisture"] = w.fuelmoisture.max()
survdata.loc[survdata["FOD_ID"]==201812071,"min_fuelmoisture"] = w.fuelmoisture.min()
survdata.loc[survdata["FOD_ID"]==201812071,"max_maxtemp"] = w.maxtemp.max()
survdata.loc[survdata["FOD_ID"]==201812071,"min_mintemp"] = w.mintemp.min()
survdata.loc[survdata["FOD_ID"]==201812071,"max_maxhumid"] = w.maxhumid.max()
survdata.loc[survdata["FOD_ID"]==201812071,"min_minhumid"] = w.minhumid.min()
survdata.loc[survdata["FOD_ID"]==201812071,"sum_precip_duration"] = w.precip_duration.sum()
survdata.loc[survdata["FOD_ID"]==201812071,"sum_precip_amt"] = w.precip_amt.sum()
survdata.loc[survdata["FOD_ID"]==201812071,"med_precip_amt"] = w.precip_amt.median()

In [265]:
survdata[survdata["FOD_ID"]==201812071]

,FOD_ID,FIPS_NAME,DATE,NEXTDATE,SurvivalTime,FireIncidence,atm_moisture,wind_azimuth,avg_windspeed,fuelmoisture,...,max_maxtemp,min_mintemp,max_maxhumid,min_minhumid,sum_precip_duration,sum_precip_amt,med_precip_amt,max_drybulbtemp,min_drybulbtemp,min_avg_windspeed
165016,201812071,NaN,2013-03-13,2013-03-17,4.0,1,NaN,NaN,NaN,NaN,...,73.2,44.5,76.0,25.4,0.0,0.0,0.0,70.1,68.9,3.2


In [269]:
fire

,FOD_ID,FPA_ID,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,FIPS_CODE,FIPS_NAME,DATE,MONTH
0,1,FS-1418826,2005,2453403.5,33,1300.0,9,Miscellaneous,0.10,A,40.036944,-121.005833,CA,63,Plumas,2005-02-02,2
1,2,FS-1418827,2004,2453137.5,133,845.0,1,Lightning,0.25,A,38.933056,-120.404444,CA,61,Placer,2004-05-12,5
2,3,FS-1418835,2004,2453156.5,152,1921.0,5,Debris Burning,0.10,A,38.984167,-120.735556,CA,17,El Dorado,2004-05-31,5
3,4,FS-1418845,2004,2453184.5,180,1600.0,1,Lightning,0.10,A,38.559167,-119.913333,CA,3,Alpine,2004-06-28,6
4,5,FS-1418847,2004,2453184.5,180,1600.0,1,Lightning,0.10,A,38.559167,-119.933056,CA,3,Alpine,2004-06-28,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189545,300348363,2015CAIRS29019636,2015,2457291.5,269,1726.0,13,Missing/Undefined,0.01,A,40.481637,-122.389375,CA,89,Shasta,2015-09-26,9
189546,300348373,2015CAIRS29217935,2015,2457300.5,278,126.0,9,Miscellaneous,0.20,A,37.617619,-120.938570,CA,99,Stanislaus,2015-10-05,10
189547,300348375,2015CAIRS28364460,2015,2457144.5,122,2052.0,13,Missing/Undefined,0.10,A,37.617619,-120.938570,CA,99,Stanislaus,2015-05-02,5
189548,300348377,2015CAIRS29218079,2015,2457309.5,287,2309.0,13,Missing/Undefined,2.00,B,37.672235,-120.898356,CA,99,Stanislaus,2015-10-14,10


In [278]:
len(survdata["FOD_ID"].unique())

6799

In [322]:
fire.FIPS_CODE.unique()

array([  63,   61,   17,    3,    5,   89,   93,   23,   37,   51,   65,
         19,   29,  111,   53,  109,   83,   79,   91,  107,   71,   73,
         33,   35,  103,    7,   49,  105,   27,   57,   15,  115,   45,
         21,   43,    9,   59,   39,   11,   99, 2031, 2005, 2019,   31,
       2021, 2510, 5033,   97,   25, 4012, 4027, 4015,   67,   47,   69,
        101,    1,   85,   13,  113,   55,   75,   81,   41,   87,   95,
       5035,   77], dtype=int64)